In [100]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib 
import datetime
from openpyxl import load_workbook

Using matplotlib backend: Qt5Agg


In [115]:
#subjectID = 'tb' 
subjectID = 'jph'


if subjectID == 'jph':
    nextSubj = ('tb',)
    painTrial_condition = ['right', 'noPain1', 'left', 'noPain2']
else:
    painTrial_condition = ['noPain1', 'left', 'noPain2','right']

In [106]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [107]:
def select_participants(cur, subjectID):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """

    cur.execute("SELECT StartTime FROM TestBlocks WHERE ParticipantCode = ?", (subjectID,))
    TimeStart = cur.fetchall()
    
    cur.execute("SELECT TestBlockId FROM TestBlocks WHERE ParticipantCode = ?", (subjectID,))
    IndexStart = cur.fetchall()
    
    
    
        
    return TimeStart, IndexStart

In [108]:
def select_experiment_data(cur, timeStart, timeEnd):

    #print(timeStart, timeEnd)
    cur.execute("SELECT TimeStamp, GazePositionX, GazePositionY, PupilDiameterLeft, PupilDiameterRight FROM DataLogs WHERE TimeStamp >= ? AND TimeStamp < ? ", (timeStart,timeEnd))
    #cur.execute('SELECT * FROM DataLogs WHERE TimeStamp BETWEEN "08/09/2019 11:37:55.304" AND "08/09/2019 11:42:13.930"')
    data = cur.fetchall()
    
    return data

In [109]:
# function to convert list of date and time into datetime format list
def timeConversion_VR(timeStrList):
    timeList = list()
    for time in timeStrList:
        timeList.append(datetime.datetime.strptime(time[0:-3], "%d/%m/%Y %H:%M:%S.%f"))
    return timeList

In [110]:
def filterBlinks_pain(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['TimeStamp', 'PupilDiameterLeft', 'PupilDiameterRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    #extraBlinkSamples = 22   
    extraBlinkSamples = 0
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single_left = np.argwhere(np.isnan(pupilDataL)).tolist()
    missingVal_Single_right = np.argwhere(np.isnan(pupilDataR)).tolist()
    
    missingVal_Single = np.unique(np.array(sorted([index for list_ in [missingVal_Single_left, missingVal_Single_right] for index in list_])))
    
    #missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    missingVal_Single_remove = [val for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if (not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])) or ((not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])))]
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        print('no nan values present')
        interpolatedNan = np.array([False]*len(pupilData_df['PupilDiameterLeft']))
        return pupilData_df, interpolatedNan
    
    # find the index and values to replace for single nan values
    #pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    #pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    
    
    interpolatedNan = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        #print(ind, len(pupilData_df['PupilDiameterRight']))
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('PupilDiameterLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('PupilDiameterRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['PupilDiameterLeft']
    missingVal_Rest_trueFalse_left = pupilData_df['PupilDiameterLeft'].isnull()
    missingVal_Rest_trueFalse_right = pupilData_df['PupilDiameterRight'].isnull()
    missingVal_Rest_left = [i for i, x in enumerate(missingVal_Rest_trueFalse_left) if x]
    missingVal_Rest_right = [i for i, x in enumerate(missingVal_Rest_trueFalse_right) if x]
    
    
    missingVal_Rest = np.unique(np.array(sorted([index for list_ in [missingVal_Rest_left, missingVal_Rest_right] for index in list_])))
 
    #missingVal_Rest_trueFalse = sorted(missingVal_Rest_trueFalse_left + list(set(missingVal_Rest_trueFalse_right)) - set(missingVal_Rest_trueFalse_left))
    
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        print('no blinks after single nan values removed')
        return pupilData_df, interpolatedNan
    
        
    
    
    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['PupilDiameterLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['PupilDiameterLeft'])-1) else (len(pupilData_df['PupilDiameterLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        print('blinks being combined')
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'PupilDiameterLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'PupilDiameterRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan[replaceTrueList] = True
         
    pupilData_df['PupilDiameterLeft'] = pupilData_df['PupilDiameterLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['PupilDiameterRight'] = pupilData_df['PupilDiameterRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan

In [111]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec


In [117]:
database = r"C:\DTU\Data\PainExperiments\20190812_PainTrial5_VR\TestResults.db"

a = list()
# create a database connection
conn = create_connection(database)

print('subject: ', subjectID)

with conn:
    # obtain the time of start of each experiment with the participant selected
    print("Obtain participant conditions")
    cur = conn.cursor()
    timeStartList, IndexStartList = select_participants(cur, subjectID)
    
    # obtain the end time of each trial
    IndexEndList = np.array([num for list_ in IndexStartList for num in list_])+1

    for i, indE in enumerate(IndexEndList):
        timeStart = timeStartList[i]
        index = (str(indE),)
        cur.execute("SELECT StartTime FROM TestBlocks WHERE TestBlockId = ?", index)
        timeEnd = cur.fetchone()
        
        if timeEnd is None:
            cur.execute("SELECT TimeStamp FROM DataLogs ORDER BY DataLogId DESC LIMIT 1")
            timeEnd = cur.fetchone()
        
        # now obtain data required for each experiment by the participant
        dataExpt = select_experiment_data(cur, timeStart[0][0:-3], timeEnd[0][0:-3])
        
        condition = painTrial_condition[i]
        
        df = pd.DataFrame(dataExpt, columns=['TimeStamp', 'GazePositionX', 'GazePositionY', 'PupilDiameterLeft', \
                                             'PupilDiameterRight'])
        # convert timestamp into the datetime format
        df['TimeStamp'] = timeConversion_VR(df['TimeStamp'])
        
        # replace 0 with nan
        df['PupilDiameterLeft'][df['PupilDiameterLeft'] < 0.003] = np.nan
        df['PupilDiameterRight'][df['PupilDiameterRight'] < 0.003] = np.nan
        #df['PupilDiameterLeft'] = df['PupilDiameterLeft'].apply(lambda x: [x if float(x) > 0.003 else np.nan])
    
        # filter the data
        df_woBlink, interPolated_woBlink = filterBlinks_pain(df['PupilDiameterLeft'], df['PupilDiameterRight'], \
                                                             df['TimeStamp'])
        
        df_filtered = df_woBlink.copy()
        df_filtered['PupilDiameterLeft_hampel'] = hampel(df_woBlink['PupilDiameterLeft'], 25, 3, False)
        df_filtered['PupilDiameterRight_hampel'] = hampel(df_woBlink['PupilDiameterRight'], 25, 3, False)
        
        # plot the data
        plt.figure()
        plt.plot(df_filtered['TimeStamp'], df_filtered['PupilDiameterLeft_hampel'], label = 'left pupil')
        plt.plot(df_filtered['TimeStamp'], df_filtered['PupilDiameterRight_hampel'], label = 'right pupil')
        
        plt.legend()
        plt.title('{} trial with subject: {}, with mean, standard deviation : right {}, {} and left {}, {}'\
                  .format(condition, subjectID, np.round(np.mean(df_filtered['PupilDiameterLeft_hampel']), 5), \
                    np.round(np.std(df_filtered['PupilDiameterLeft_hampel']), 5), \
                    np.round(np.mean(df_filtered['PupilDiameterRight_hampel']), 5), \
                    np.round(np.std(df_filtered['PupilDiameterRight_hampel']), 5)))
        
        
        
        plt.xlabel('Time')
        plt.ylabel('Pupil diameter [in mm]')
        plt.ylim([0.005, 0.0068])
        
        a.append([np.mean(df_filtered['PupilDiameterLeft_hampel']), np.std(df_filtered['PupilDiameterLeft_hampel']), \
                    np.mean(df_filtered['PupilDiameterRight_hampel']), np.std(df_filtered['PupilDiameterRight_hampel'])])
        
        #a.append([np.min(df_filtered['PupilDiameterLeft_hampel']), np.max(df_filtered['PupilDiameterLeft_hampel']), \
        #            np.min(df_filtered['PupilDiameterRight_hampel']), np.max(df_filtered['PupilDiameterRight_hampel'])])
        
        
        
        pupilDiameterLeft = dict()
        pupilDiameterLeft[condition+'_meanLeft'] = np.mean(df_filtered['PupilDiameterLeft_hampel'])
        pupilDiameterLeft[condition+'_stdLeft'] = np.std(df_filtered['PupilDiameterLeft_hampel'])
        pupilDiameterLeft[condition+'_meanRight'] = np.mean(df_filtered['PupilDiameterRight_hampel'])
        pupilDiameterLeft[condition+'_stdRight'] = np.std(df_filtered['PupilDiameterRight_hampel'])
        
        #print(pupilDiameterLeft)
        
        
resultPathName = r'C:\DTU\Results\20190812_PainTrial5_VR\pupilSize_overview.xlsx'
dataFrame = pd.DataFrame.from_records(a)
print(dataFrame)
book = load_workbook(resultPathName)
writer = pd.ExcelWriter(resultPathName, engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
startrow = writer.sheets['Sheet1'].max_row
dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
writer.save() 

subject:  jph
Obtain participant conditions


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


          0         1         2         3
0  0.005653  0.000210  0.006062  0.000228
1  0.005687  0.000225  0.006068  0.000154
2  0.005777  0.000203  0.006149  0.000176
3  0.005699  0.000164  0.005994  0.000149


In [73]:
np.round(np.mean(df_filtered['PupilDiameterLeft_hampel']), 5)

0.00594